# How do we use interpolative decomposition and QR algorithms to prune neural nets in practice?  

Now we can use our interpolative decomposition to prune our network.  First, we are going to train a network with 1000 hidden layer neurons on our digit classification task from the first tutorial.  


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import datasets
from copy import deepcopy
import scipy.linalg
#load the dataset from the library.  
digits = datasets.load_digits()

n_samples = len(digits.images)
    
#flatten the images to a single column of 64.     
data = digits.images.reshape((n_samples, -1))


X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.5, shuffle=False)


clf = MLPClassifier(1000,random_state=1, max_iter=300).fit(X_train, y_train)
print("Training Accuracy: {}%".format(round(clf.score(X_train, y_train)*100,2)))
print("Test Accuracy: {}%".format(round(clf.score(X_test, y_test)*100,2)))

Note that this performs better on the data than our original (smaller) network. In the very over-parameterized case, the network has enough parameters to completely memorize all of the data points in the training set.  In more complicated neural networks, especially in deep learning, the networks have more than enough parameters to memorize all of the training data.  However, they often do not over-fit to the extent that we might expect they would.  They can, in fact, start to "generalize" - meaning perform better than smaller nets on unseen data. 

This phenomenon is called double descent. Let's think about curve fitting: there is an old saying attributed to von Neumann, "with four parameters I can fit an elephant, and with five I can make him wiggle his trunk".  Fitting all of the data perfectly is called interpolation.  When we have the same number of parameters as we have data points, we can interpolate all of the points perfectly.  As anyone who has done curve fitting probably knows, this function is usually wild and does not fit the data well off of the data points.  However, as you keep adding more parameters, the fitting function has more different choices of parameter values that could fit the data.  When it has many choices of possible solutions, in some cases (particularly with under-determined least squares problems), we have reason to expect that it will pick the simplest choice, the minimal norm. 


There is also the idea of the "Lottery Ticket Hypothesis".  This is the idea that for some large enough network, there is a smaller subnet inside it that could train to perform very well on the data by itself.  However, it is empirically difficult to train small networks.  In general, the fewer parameters there are, the more the parameters have to move to perfectly classify all of the data.  Overall, in practice, networks are used that are vastly overparameterized.  Networks use many hidden layers, and can include millions of parameters.  

Therefore, the networks that are made in practice often could be "pruned".  This means that we reduce the number of neurons in each layer.  For our example, we reduce the number of neurons in the one hidden layer.  We need to pick which neurons to eliminate, and compensate somehow.  Let's take a look at our matrices from tutorial 1.   

We have several choices for pruning.  We could try to just prune the weight matrix $W$, preserving the largest neurons (or largest columns in the weight matrix).  This is called magnitude pruning.  This is one of the more commonly used methods.  However, this does not use any information that we have from our data.  We take a look at how this performs below, when pruning half of the neurons.    



In [ ]:
pruned=deepcopy(clf)  #want to keep original model.  
W=pruned.coefs_[0]
b=pruned.intercepts_[0]


norms=np.sum(-W**2, axis=0)

indx=np.argsort(norms)[0:500]

W2=W[:,indx]

b2=b[indx]
pruned.coefs_[0]=W2
pruned.intercepts_[0]=b2
pruned.coefs_[1]=pruned.coefs_[1][indx,:]
print("Pruned Training Accuracy: {}%".format(round(pruned.score(X_train, y_train)*100,2)))
print("Pruned Test Accuracy: {}%".format(round(pruned.score(X_test, y_test)*100,2)))

However, we want to include information from the data.  We can do that by choosing neurons so that we approximate the matrix $Z = ReLU(xW+b)$.  This also allows our pruning algorithm to have information about the activation function.  

In [ ]:
pruned=deepcopy(clf)
W=pruned.coefs_[0]
print(W.shape)
b=pruned.intercepts_[0]
U=pruned.coefs_[1]
k=500 #number of neurons kept

Z=np.maximum(X_train@W+b, 0)
print("shape of Z")
print(Z.shape)
print("Z:")
print(Z)

The whole network can be described by the equation:  $\sigma ( ZU + \beta)$

We are going to sub-select some columns of $Z$, and then update $U$ (the coefficients in the next layer) so that we can compensate for the columns that are missing.  By sub-selecting columns of $Z$, we are only using the contributions of certain neurons.  Therefore, we can prune away the rest.  This commutes with the activation function.  


We take the interpolative decomposition,  $Z\approx  Z_{I,:} T$.  $T$ is our interpolating matrix, and $Z_{I,:}$ is the sub-set of columns of $Z$.  We replace $Z$ in our equation:  

$\sigma ( Z_{I,:} T U+ \beta)$

Note that $T$ is just a matrix.  We can multiply on the right by $U$.  This is equivalent to updating the coefficients in the next layer to compensate for the missing neurons.  

Now, we can use our pivoted QR decomposition from the third tutorial to calculate $T$.    

$T=[I_k, R_{11}^{-1}R_{12}]\Pi ^T$




In [ ]:
R, Pi = scipy.linalg.qr((Z), mode='r', pivoting=True)

T = np.concatenate((
        np.identity(k),
        np.linalg.pinv(R[0:k, 0:k]) @ R[0:k, k:None]
        ), axis=1)
T = T[:, np.argsort(Pi)]

print(T.shape)

Then we simply multiply $U$ by $T$ to update the next layer.  

In [ ]:
print(U.shape)
nextLayer=T@U
print(nextLayer.shape)
pruned.coefs_[1]=nextLayer

Since we have sub-selected columns of $Z$, we can sub-select the neurons that made $Z$.  This corresponds to sub-selecting columns in $W$.  

In [ ]:
print(W.shape)
Wk = W[:, Pi[0:k]]
bk=b[Pi[0:k]]
pruned.coefs_[0]=Wk
pruned.intercepts_[0]=bk

Let's see how it does!  

In [ ]:
print("Pruned Training Accuracy: {}%".format(round(pruned.score(X_train, y_train)*100,2)))
print("Pruned Test Accuracy: {}%".format(round(pruned.score(X_test, y_test)*100,2)))

Now we have pruned half of the neurons, and kept most of the accuracy!  Let's compare this to a model of the same size run from scratch.  

In [ ]:
clfSmall = MLPClassifier(k,random_state=1, max_iter=300).fit(X_train, y_train)
print("Smaller Model Training Accuracy: {}%".format(round(clfSmall.score(X_train, y_train)*100,2)))
print("Smaller Model Test Accuracy: {}%".format(round(clfSmall.score(X_test, y_test)*100,2)))

While this is a single trial, and not statistically significant, we have shown that we can prune a larger network and keep most of the accuracy in a way that can be better than or competitive with a smaller model trained from scratch!

Finally, let's look at the accuracy of the model.  We can actually make some guarantees about how much our pruning method will change what comes out.  

Remember that the error on our interpolative decomposition as calculated by the column-pivoted QR was $||R_{22}||$. 

Let's check that below.  

In [ ]:
print(scipy.linalg.norm(R[k:None, k:None], ord=2))

Z=np.maximum(X_train@W+b, 0)
Zprime=np.maximum(X_train@Wk+bk, 0)
Zprime=(Zprime@T)

print(scipy.linalg.norm(Z-Zprime, ord=2))


In the worst possible case, this error could be exactly aligned with $U$.  Therefore, we can use the Cauchy-Schwartz inequality:  $||R_{22}U||\leq||U||\ ||R_{22}||$

Again, we can check this below.  



In [ ]:
Uz=Z@U
UZprime=Zprime@U
print("norm of R22 U")
print(scipy.linalg.norm(Uz-UZprime, ord=2))
print("norm of R22 times norm of U")
print(scipy.linalg.norm(U, ord=2)*scipy.linalg.norm(R[k:None, k:None], ord=2))

Since the error $||R_{22}U||$ is much smaller than the maximum error, we have found that we do much better than the worst case!  

Overall, we can easily extend this method to multi-hidden-layer networks by writing them as multiplications of matrices, and then applying interpolative decompositions.  Convolutional networks (which are more commonly used for images) can be more difficult, but are doable.  

Finally, let's compare the compute times for each network:  



In [ ]:
import time
start=time.time()
clf.score(X_train, y_train)
print("Time for big network:  {}s".format(time.time()-start))
start=time.time()
pruned.score(X_train, y_train)
print("Time for pruned network:  {}s".format(time.time()-start))

We have significantly reduced the time for our network to classify data points by pruning it and reducing the number of parameters!  